# Search for long range edges 

In this notebook, I will implement code to search for a long range edge(s) from a default `feems` by seeing if the likelihood increases with the addition of a certain edge. First, the search for the *best* edge will be done through a full grid search, wherein I iteratively add one long range edge after another over all pairs of nodes and check if this extra edge decreases the negative log likelihood. Second, I will implement a heuristic search by using a greedy approach to fit an edge between a pair of nodes that show maximum residuals with the default fit. 

To do this, I need to simulate a large empirical test case with many nodes. I will use the same corridor-barrier-corridor approach from previous simulations but with a much larger grid (say, 40x80). 

## Changes to original code base

1. added code to calculate the negative log likelihood value for fit in `spatial_graph.py` (basically, adding `Objective` functions)

## Imports 

In [71]:
%load_ext autoreload
%autoreload 2

# base
import numpy as np
import networkx as nx
from sklearn.impute import SimpleImputer
import pkg_resources
import itertools as it
import math
from scipy.spatial.distance import pdist, squareform
import statsmodels.api as sm
from copy import deepcopy
import pandas as pd

# viz
import matplotlib.pyplot as plt
from matplotlib import gridspec
import cartopy.crs as ccrs

# feems
from feems.utils import prepare_graph_inputs
from feems import SpatialGraph, Viz, Objective
from feems.sim import setup_graph, setup_graph_long_range, simulate_genotypes
from feems.spatial_graph import query_node_attributes
from feems.objective import comp_mats
from feems.cross_validation import run_cv
from feems.helper_funcs import plot_default_vs_long_range, comp_genetic_vs_fitted_distance, plot_estimated_vs_simulated_edges

# change matplotlib fonts
plt.rcParams["font.family"] = "Arial"
plt.rcParams["font.sans-serif"] = "Arial"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Simulation test case


In [23]:
n_rows, n_columns = 4, 8
graph_def, _, _, edge_def = setup_graph(n_rows=n_rows, n_columns=n_columns, barrier_startpt=2.5, barrier_endpt=5.5, corridor_w=0.5, barrier_w=0.1, barrier_prob=1.0)

lrn = [(15,20)]

## using 1.0 to ensure all nodes are sampled equally well (default params otherwise: 4x8 grid)
graph, coord, grid, edge = setup_graph_long_range(n_rows=n_rows, n_columns=n_columns, corridor_w=1.0, barrier_w=0.5, barrier_prob=1.0, long_range_nodes=lrn, long_range_edges=[2.0])

gen_test = simulate_genotypes(graph)

Simulating ~SNP 0
Simulating ~SNP 50
Simulating ~SNP 100
Simulating ~SNP 200
Simulating ~SNP 250
Simulating ~SNP 350
Simulating ~SNP 400
Simulating ~SNP 450
Simulating ~SNP 500
Simulating ~SNP 650
Simulating ~SNP 700
Simulating ~SNP 750
Simulating ~SNP 800
Simulating ~SNP 900
Simulating ~SNP 950


In [24]:
sp_Graph_def = SpatialGraph(gen_test, coord, grid, edge_def)

In [77]:
# create a list of all edges to add (since it is symmetric, we should expect d(d-1)/2 but some will be adjacent nodes so fewer than that)
lr = (tuple(i) for i in it.product(tuple(range(sp_Graph_def.n_observed_nodes)), repeat=2) if tuple(reversed(i)) > tuple(i))
final_lr = [x for x in list(lr) if x not in list(sp_Graph_def.edges)]

In [78]:
df = pd.DataFrame(index = np.arange(len(final_lr)), columns = ['nodes', 'nll'])

In [ ]:
%%time
for idx, val in enumerate(final_lr):
    # creating a new edge vector (all default edges + 1 long range)
    edges_lr = deepcopy(edge_def)
    edges_lr = edges_lr.tolist()
    edges_lr.append(list(x+1 for x in val))
    sp_Graph = SpatialGraph(gen_test, coord, grid, np.array(edges_lr))
    sp_Graph.fit(lamb = 10.0, verbose=False)

    df.iloc[idx, 0] = val
    df.iloc[idx, 1] = sp_Graph.nll

# print nodes connected by THE edge to give lowest negative log likelihood
df.loc[df['nll'].astype(float).idxmin(),'nodes']